In [ ]:
from P2 import *
import numpy as np
import os, math, time, argparse
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
data, labels, sub_info, form_names, feat_names  = load_dataset(data_path = "P2/data")
num_subs = len(np.unique(sub_info[:, 0]))
num_feats = data.shape[1]
num_forms = len(form_names) + 1
filter_feat_count = 100

In [ ]:
sub_train_acc = np.zeros(num_subs)
sub_class_train = np.zeros((num_subs, num_forms))
sub_test_acc = np.zeros(num_subs)
sub_class_test = np.zeros((num_subs, num_forms))
overall_train_mat = np.zeros((num_forms, 1))
overall_test_mat = np.zeros((num_forms, 1))

In [ ]:
train_data, train_label, test_data, test_label = split_data(data, labels, sub_info, 1)
train_data, train_min, train_max = normalize_data(train_data)
test_data, test_min, test_max = normalize_data(test_data)

In [ ]:
def filter_method(train_data, train_labels):
    scores = np.zeros(train_data.shape[1])
    train_label_unique = np.unique(train_labels)
    for feature in range(train_data.shape[1]):
        sum_per_label_var = 0
        var_feature = np.var(train_data[:, feature])
        for label in train_label_unique:
            train_label_list = train_data[:,feature][train_labels == label]
            train_label_var = np.var(train_label_list)
            sum_per_label_var+=train_label_var
        complete_sum_label = sum_per_label_var/len(train_label_unique)
        if complete_sum_label!=0:
            scores[feature] = var_feature/complete_sum_label
    selected_inds = np.argsort(scores)[::-1]
    return selected_inds, scores

In [ ]:
sc, scr = filter_method(train_data, train_label)

In [ ]:
train_data = train_data[:, sc[:100]]
test_data = test_data[:, sc[:100]]

In [ ]:
selected_inds = np.array([], dtype = np.int16)
model = LinearDiscriminantAnalysis()
value = 0.8
for i in range(train_data.shape[1]):
    local_score = 0
    temp = 0
    for j in range(train_data.shape[1]):
        # if local_score>value:
        #     return selected_inds
        if j in selected_inds:
            continue
        t_d = np.hstack((train_data[:, selected_inds], train_data[:, j].reshape(-1, 1)))
        model.labels_ = np.unique(train_label)
        model.fit(t_d, train_label)
        sub_pred_train = model.predict(t_d)
        train_acc = np.sum(sub_pred_train == train_label) / len(train_label)
        if train_acc > local_score:
            temp = j
            local_score = train_acc
        selected_inds = np.append(selected_inds, temp)